In [1]:
import os
import sys
import pandas as pd
from IPython.display import display, HTML

# setting warnings
import warnings
warnings.simplefilter(action='ignore', category = Warning)

# add parent folder path to the namespace
sys.path.append(os.path.dirname(os.getcwd()))

# import modules and components
from utils.data_assets import PreProcessing
import utils.global_paths as globpt
import configurations as cnf

# specify relative paths from global paths and create subfolders
mat_path = os.path.join(globpt.data_path, 'materials') 
exp_path = os.path.join(globpt.data_path, 'experiments') 
os.mkdir(mat_path) if not os.path.exists(mat_path) else None
os.mkdir(exp_path) if not os.path.exists(exp_path) else None 

### Load and prepare data

In [2]:
filepath = os.path.join(mat_path, 'adsorbents_dataset.csv')  
df_adsorbents = pd.read_csv(filepath, sep=';', encoding='utf-8')  
filepath = os.path.join(mat_path, 'adsorbates_dataset.csv')  
df_adsorbates = pd.read_csv(filepath, sep=';', encoding='utf-8')  
filepath = os.path.join(exp_path, 'single_component_dataset.csv')  
df_SCADS = pd.read_csv(filepath, sep=';', encoding='utf-8')
filepath = os.path.join(exp_path, 'binary_mixture_dataset.csv')  
df_BMADS = pd.read_csv(filepath, sep=';', encoding='utf-8')

## 1. Data validation

BMADS dataset is not analyzed fully, as it is of less interest. Some general info on BMADS can be found here:

In [4]:
num_BMADS_experiments = df_BMADS['filename'].nunique()
print('\nBMADS dataset: binary mixture adsorption')
print('-----------------------------------------------')
print(f'Number of measurements: {df_BMADS.shape[0]}')
print(f'Number of actual experiments: {num_BMADS_experiments}')
print(f'Average measurements per experiment: {df_BMADS.shape[0]//num_BMADS_experiments}')


BMADS dataset: binary mixture adsorption
-----------------------------------------------
Number of measurements: 20403
Number of actual experiments: 2372
Average measurements per experiment: 8


### 1.1 Analyze raw dataset

SCADS dataset (single component adsorption) is analyzed and validate. The major aim is to provide a fully preprocessed dataset for machine learning. THe information reported in this cell refer to the raw dataset

In [5]:
preprocessor = PreProcessing()

num_SCADS_experiments = df_SCADS['filename'].nunique()
SCADS_negative_press = df_SCADS[df_SCADS['pressure'] < 0].shape[0]
SCADS_negative_uptake = df_SCADS[df_SCADS['adsorbed_amount'] < 0].shape[0]

print('\nSCADS dataset: single component adsorption')
print('-----------------------------------------------')
print(f'Number of measurements: {df_SCADS.shape[0]}')
print(f'Number of actual experiments: {num_SCADS_experiments}')
print(f'Average measurements per experiment: {df_SCADS.shape[0]//num_SCADS_experiments}\n')
print(f'Negative pressure measurements: {SCADS_negative_press} ({SCADS_negative_press/df_SCADS.shape[0] * 100:.2f}%)')
print(f'Negative uptake measurements: {SCADS_negative_uptake} ({SCADS_negative_uptake/df_SCADS.shape[0] * 100:.2f}%)')
print(f'Negative uptake measurements: {SCADS_negative_uptake} ({SCADS_negative_uptake/df_SCADS.shape[0] * 100:.2f}%)')


SCADS dataset: single component adsorption
-----------------------------------------------
Number of measurements: 610667
Number of actual experiments: 28606
Average measurements per experiment: 21

Negative pressure measurements: 737 (0.12%)
Negative uptake measurements: 1654 (0.27%)


### 1.2 Preprocess SCADS dataset

In [ ]:
# add molecular properties based on PUGCHEM API data
print('Adding physicochemical properties from guest species dataset\n')
dataset = preprocessor.add_guest_properties(df_SCADS, df_adsorbates)
dataset = dataset.dropna()

In [ ]:

# filter experiments leaving only valid uptake and pressure units, then convert 
# pressure and uptake to Pa (pressure) and mol/kg (uptake)

# filter experiments by pressure and uptake units 
dataset = dataset[dataset[preprocessor.Q_unit_col].isin(preprocessor.valid_units)]

# convert pressures to Pascal
dataset[preprocessor.P_col] = dataset.progress_apply(lambda x : preprocessor.pressure_converter(x[preprocessor.P_unit_col], 
                                                                                                x['pressure']), 
                                                                                                axis = 1)
# convert uptakes to mol/g
dataset[preprocessor.Q_col] = dataset.progress_apply(lambda x : preprocessor.uptake_converter(x[preprocessor.Q_unit_col], 
                                                                                              x['adsorbed_amount'], 
                                                                                              x['mol_weight']), 
                                                                                              axis = 1)

# further filter the dataset to remove experiments which values are outside desired boundaries, 
# such as experiments with negative temperature, pressure and uptake values
#------------------------------------------------------------------------------ 
dataset = dataset[dataset['temperature'].astype(int) > 0]
dataset = dataset[dataset[preprocessor.P_col].astype(float).between(0.0, cnf.max_pressure)]
dataset = dataset[dataset[preprocessor.Q_col].astype(float).between(0.0, cnf.max_uptake)]

## 2. Data analysis

### 1.1 Explore materials datasets

In [ ]:
unique_adsorbents = df_adsorbents['name'].nunique()
unique_adsorbates = df_adsorbates['name'].nunique()

print(f'\nNumber of adsorbents in dataset: {unique_adsorbents}')
print(f'Number of adsorbates in dataset: {unique_adsorbates}')

### 1.2 Explore experiments datasets